# CRPBot Model Evaluation - Clean Setup

**Models**: 3 LSTM (BTC, ETH, SOL) - 128/3/bidirectional, 50 features  
**Goal**: Evaluate against promotion gates (≥68% accuracy, ≤5% calibration error)  
**Time**: ~5-10 minutes on GPU

## Step 1: Check GPU

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not available! Go to Runtime → Change runtime type → GPU")

## Step 2: Install Dependencies

In [ ]:
!pip install -q loguru pandas pyarrow scikit-learn

## Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify files exist
import os
drive_path = '/content/drive/MyDrive/colab'

print("\n📂 Checking files...")
print(f"\nDrive path exists: {os.path.exists(drive_path)}")

if os.path.exists(drive_path):
    print("\nContents:")
    !ls -lh /content/drive/MyDrive/colab/
else:
    print("❌ ERROR: /content/drive/MyDrive/colab/ not found!")
    print("Please ensure your files are uploaded to 'My Drive/colab/' folder")

## Step 4: Copy Files to Colab Workspace

In [ ]:
# Create directories
!mkdir -p models/new data/features

# Copy model files
print("📦 Copying model files...")
!cp /content/drive/MyDrive/colab/models/*.pt models/new/ 2>/dev/null || \
 cp /content/drive/MyDrive/colab/*.pt models/new/ 2>/dev/null || \
 echo "⚠️ No .pt files found in expected locations"

# Copy feature files
print("\n📦 Copying feature files...")
!cp /content/drive/MyDrive/colab/features/*.parquet data/features/ 2>/dev/null || \
 cp /content/drive/MyDrive/colab/*.parquet data/features/ 2>/dev/null || \
 echo "⚠️ No .parquet files found in expected locations"

# Verify
print("\n✅ Files in workspace:")
print("\nModels:")
!ls -lh models/new/
print("\nFeatures:")
!ls -lh data/features/

# Check counts
import glob
models = glob.glob('models/new/*.pt')
features = glob.glob('data/features/*.parquet')

print(f"\n📊 Summary:")
print(f"  Models found: {len(models)}/3 expected")
print(f"  Features found: {len(features)}/3 expected")

if len(models) != 3 or len(features) != 3:
    print("\n⚠️ WARNING: Missing files! Expected 3 models + 3 features")
    print("Please check your Google Drive folder structure.")

## Step 5: Create Code Modules

In [ ]:
!mkdir -p apps/trainer/{models,train,eval} libs/{constants,rl_env}

In [ ]:
%%writefile apps/trainer/models/lstm.py
"""LSTM model for direction prediction."""
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    """Bidirectional LSTM for binary classification."""

    def __init__(
        self,
        input_size: int,
        hidden_size: int = 128,
        num_layers: int = 3,
        dropout: float = 0.2,
        bidirectional: bool = True,
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional,
        )

        lstm_output_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc1 = nn.Linear(lstm_output_size, 64)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        x = torch.relu(self.fc1(last_output))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x)

In [ ]:
%%writefile apps/trainer/data_pipeline.py
"""Data loading and preprocessing."""
from pathlib import Path
from typing import Tuple

import pandas as pd
from loguru import logger

def load_features_from_parquet(symbol: str, features_dir: str = "data/features") -> pd.DataFrame:
    """Load features from parquet."""
    # Try multiple patterns
    patterns = [
        f"features_{symbol}_1m_latest.parquet",
        f"features_{symbol}_1m_*_50feat.parquet",
        f"features_{symbol}_*.parquet",
    ]
    
    for pattern in patterns:
        files = list(Path(features_dir).glob(pattern))
        if files:
            features_path = sorted(files)[-1]  # Use latest
            df = pd.read_parquet(features_path)
            logger.info(f"Loaded features from {features_path}: {len(df):,} rows, {len(df.columns)} columns")
            return df
    
    raise FileNotFoundError(f"No feature file found for {symbol} in {features_dir}")

def create_walk_forward_splits(
    df: pd.DataFrame,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Split data chronologically."""
    n = len(df)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    
    train_df = df.iloc[:train_end].copy()
    val_df = df.iloc[train_end:val_end].copy()
    test_df = df.iloc[val_end:].copy()
    
    logger.info(f"Creating walk-forward splits: train until {train_df['timestamp'].max()}, val until {val_df['timestamp'].max()}")
    logger.info(f"Split sizes: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    
    return train_df, val_df, test_df

In [ ]:
%%writefile apps/trainer/features.py
"""Feature normalization."""
from typing import List, Tuple
import pandas as pd
from loguru import logger

FEATURE_COLUMNS = [
    # Session (5)
    "session_tokyo", "session_london", "session_ny", "day_of_week", "is_weekend",
    # Spread (4)
    "spread", "spread_pct", "atr", "spread_atr_ratio",
    # Volume (3)
    "volume_ma", "volume_ratio", "volume_trend",
    # MA (8)
    "sma_7", "sma_14", "sma_21", "sma_50",
    "price_to_sma_7", "price_to_sma_14", "price_to_sma_21", "price_to_sma_50",
    # Technical (8)
    "rsi", "macd", "macd_signal", "macd_diff",
    "bb_upper", "bb_middle", "bb_lower", "bb_width",
    # Multi-TF OHLCV (15)
    "5m_open", "5m_high", "5m_low", "5m_close", "5m_volume",
    "15m_open", "15m_high", "15m_low", "15m_close", "15m_volume",
    "1h_open", "1h_high", "1h_low", "1h_close", "1h_volume",
    # Cross-TF (3)
    "cross_tf_alignment_score", "cross_tf_alignment_direction", "cross_tf_alignment_strength",
    # Volatility (1)
    "atr_percentile",
]

def normalize_features(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame = None,
    test_df: pd.DataFrame = None,
    method: str = "standard",
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, dict]:
    """Normalize features."""
    feature_cols = [c for c in FEATURE_COLUMNS if c in train_df.columns]
    
    if method == "standard":
        stats = {
            "mean": train_df[feature_cols].mean().to_dict(),
            "std": train_df[feature_cols].std().to_dict(),
        }
        
        for col in feature_cols:
            mean, std = stats["mean"][col], stats["std"][col]
            if std > 0:
                train_df[col] = (train_df[col] - mean) / std
                if val_df is not None:
                    val_df[col] = (val_df[col] - mean) / std
                if test_df is not None:
                    test_df[col] = (test_df[col] - mean) / std
    
    logger.info(f"Normalized {len(feature_cols)} features using {method} method")
    return train_df, val_df, test_df, stats

In [ ]:
%%writefile apps/trainer/train/dataset.py
"""PyTorch dataset."""
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from loguru import logger

class SequenceDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        feature_columns: list[str],
        sequence_length: int = 60,
        prediction_horizon: int = 15,
        prediction_type: str = "direction",
    ):
        self.df = df.reset_index(drop=True)
        self.feature_columns = feature_columns
        self.sequence_length = sequence_length
        self.prediction_horizon = prediction_horizon
        self.prediction_type = prediction_type

        self.features = self.df[feature_columns].values.astype(np.float32)
        self._precompute_targets()

        self.valid_indices = np.arange(
            sequence_length,
            len(self.df) - prediction_horizon
        )
        
        logger.info(
            f"Created dataset: {len(self.valid_indices)} sequences, "
            f"{len(feature_columns)} features, prediction_type={prediction_type}"
        )

    def _precompute_targets(self):
        future_close = self.df["close"].shift(-self.prediction_horizon)
        current_close = self.df["close"]
        
        if self.prediction_type == "direction":
            self.targets = (future_close > current_close).astype(np.float32).values
        else:
            self.targets = ((future_close - current_close) / current_close).astype(np.float32).values

    def __len__(self) -> int:
        return len(self.valid_indices)

    def __getitem__(self, idx: int):
        actual_idx = self.valid_indices[idx]
        seq_start = actual_idx - self.sequence_length
        seq_end = actual_idx
        sequence = self.features[seq_start:seq_end]
        target = self.targets[actual_idx]
        return torch.from_numpy(sequence), torch.tensor([target], dtype=torch.float32)

## Step 6: Run Evaluation

In [ ]:
%%writefile evaluate.py
"""Evaluate all 3 models on GPU."""
import sys
import numpy as np
import pandas as pd
import torch
from loguru import logger
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader

from apps.trainer.models.lstm import LSTMModel
from apps.trainer.data_pipeline import load_features_from_parquet, create_walk_forward_splits
from apps.trainer.features import normalize_features, FEATURE_COLUMNS
from apps.trainer.train.dataset import SequenceDataset

def evaluate_model(model_path: str, symbol: str, device: str = "cuda", batch_size: int = 64):
    logger.info(f"{'='*60}")
    logger.info(f"🚀 Evaluating {symbol}")
    logger.info(f"{'='*60}")
    
    # Load data
    df = load_features_from_parquet(symbol)
    train_df, val_df, test_df = create_walk_forward_splits(df)
    train_df, val_df, test_df, stats = normalize_features(train_df, val_df, test_df)
    
    # Create dataset
    feature_cols = [c for c in FEATURE_COLUMNS if c in test_df.columns]
    test_dataset = SequenceDataset(
        test_df, feature_columns=feature_cols,
        sequence_length=60, prediction_horizon=15, prediction_type="direction"
    )
    
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=2, pin_memory=(device == "cuda")
    )
    
    # Load model
    model = LSTMModel(
        input_size=len(feature_cols),
        hidden_size=128, num_layers=3,
        dropout=0.2, bidirectional=True
    )
    
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model = model.to(device)
    model.eval()
    
    logger.info(f"Evaluating on {len(test_dataset):,} sequences...")
    
    # Evaluate
    all_preds, all_targets, all_probs = [], [], []
    
    with torch.no_grad():
        for batch_idx, (sequences, targets) in enumerate(test_loader):
            sequences = sequences.to(device)
            targets = targets.to(device)
            outputs = model(sequences)
            probs = outputs.squeeze().cpu().numpy()
            preds = (probs >= 0.5).astype(int)
            
            all_probs.extend(probs.tolist() if probs.ndim > 0 else [probs.item()])
            all_preds.extend(preds.tolist() if preds.ndim > 0 else [preds.item()])
            all_targets.extend(targets.squeeze().cpu().numpy().tolist())
            
            if batch_idx % 100 == 0:
                logger.info(f"  Progress: {(batch_idx + 1) * batch_size:,}/{len(test_dataset):,}")
    
    # Metrics
    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, zero_division=0)
    recall = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds, zero_division=0)
    
    # Calibration error
    bins = np.linspace(0, 1, 11)
    bin_indices = np.digitize(all_probs, bins) - 1
    ece = 0.0
    for i in range(10):
        mask = bin_indices == i
        if mask.sum() > 0:
            bin_accuracy = np.array(all_targets)[mask].mean()
            bin_confidence = np.array(all_probs)[mask].mean()
            ece += mask.sum() / len(all_targets) * abs(bin_accuracy - bin_confidence)
    
    logger.info(f"\n✅ {symbol} Results:")
    logger.info(f"  Accuracy: {accuracy:.4f}")
    logger.info(f"  Precision: {precision:.4f}")
    logger.info(f"  Recall: {recall:.4f}")
    logger.info(f"  F1: {f1:.4f}")
    logger.info(f"  Calibration Error: {ece:.4f}")
    
    passes_acc = accuracy >= 0.68
    passes_cal = ece <= 0.05
    
    logger.info(f"\n🎯 Promotion Gates:")
    logger.info(f"  Accuracy ≥ 68%: {'✅ PASS' if passes_acc else '❌ FAIL'} ({accuracy:.2%})")
    logger.info(f"  Calibration ≤ 5%: {'✅ PASS' if passes_cal else '❌ FAIL'} ({ece:.2%})")
    logger.info(f"  Overall: {'✅ PROMOTION APPROVED' if (passes_acc and passes_cal) else '❌ NOT READY'}\n")
    
    return {
        "symbol": symbol,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "calibration_error": ece,
        "num_samples": len(all_targets),
    }

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {device}")
    
    symbols = ["BTC-USD", "ETH-USD", "SOL-USD"]
    all_results = {}
    
    for symbol in symbols:
        model_path = f"models/new/lstm_{symbol.replace('-', '_')}_1m_7b5f0829.pt"
        results = evaluate_model(model_path, symbol, device=device)
        all_results[symbol] = results
    
    logger.info(f"\n\n{'='*60}")
    logger.info("📊 EVALUATION SUMMARY")
    logger.info(f"{'='*60}")
    
    results_df = pd.DataFrame(all_results).T
    print(results_df.to_string())
    
    results_df.to_csv("evaluation_results.csv")
    logger.info(f"\n✅ Results saved to evaluation_results.csv")

In [ ]:
# Run evaluation
!python evaluate.py

## Step 7: Download Results

In [ ]:
from google.colab import files
files.download('evaluation_results.csv')

# Also display
import pandas as pd
results = pd.read_csv('evaluation_results.csv', index_col=0)
print("\n📊 Final Results:")
print(results.to_string())